In [31]:
# autoMPG
import pandas as pd
import numpy as np
from sklearnex import patch_sklearn, unpatch_sklearn
patch_sklearn()
import sklearn
import matplotlib.pyplot as plt
import matplotlib as mpl
# font to 맑은 고딕
mpl.rc('font', family='Malgun Gothic')
import seaborn as sns


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


### 데이터셋 로딩 및 전처리


In [32]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
df = pd.read_csv(url, header=None, sep='\s+')
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin', 'car_name']
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [33]:
df['origin'] = df['origin'].astype('category')
df['model_year'] = df['model_year'].apply(lambda x: pd.to_datetime(x, format='%y').year)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           398 non-null    float64 
 1   cylinders     398 non-null    int64   
 2   displacement  398 non-null    float64 
 3   horsepower    398 non-null    object  
 4   weight        398 non-null    float64 
 5   acceleration  398 non-null    float64 
 6   model_year    398 non-null    int64   
 7   origin        398 non-null    category
 8   car_name      398 non-null    object  
dtypes: category(1), float64(4), int64(2), object(2)
memory usage: 25.5+ KB


In [34]:
# df['horsepower'] = df['horsepower'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce')
df.dropna(inplace=True, axis=0)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           392 non-null    float64 
 1   cylinders     392 non-null    int64   
 2   displacement  392 non-null    float64 
 3   horsepower    392 non-null    float64 
 4   weight        392 non-null    float64 
 5   acceleration  392 non-null    float64 
 6   model_year    392 non-null    int64   
 7   origin        392 non-null    category
 8   car_name      392 non-null    object  
dtypes: category(1), float64(5), int64(2), object(1)
memory usage: 25.1+ KB


In [35]:
corrdf = df.drop([
    'car_name', 
    'origin', 
    'model_year',
    ], axis=1)
corrdf.corr()['mpg']

mpg             1.000000
cylinders      -0.777618
displacement   -0.805127
horsepower     -0.778427
weight         -0.832244
acceleration    0.423329
Name: mpg, dtype: float64

In [36]:
target = df['mpg']
features = df.drop([
    'mpg', 
    'car_name', 
    'origin' , 
    'model_year', 
    # 'acceleration',
    ], axis=1
                   )
features.tail()

,cylinders,displacement,horsepower,weight,acceleration
387,4,140.0,86.0,2790.0,15.6
388,4,97.0,52.0,2130.0,24.6
389,4,135.0,84.0,2295.0,11.6
390,4,120.0,79.0,2625.0,18.6
391,4,119.0,82.0,2720.0,19.4


In [42]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

# num = 750
saved = 0
scores_test = []
scores_train = []
# randomforest 59  n_esti 750 
# knn 59 n_neighbors = 5

# for num in range(1,300):
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    target, 
    test_size=0.2, 
    random_state=59
)

scalerList = [
        ('RBscaler', RobustScaler()),
        ('stdScaler', StandardScaler()),
        ('scalerZ', QuantileTransformer(output_distribution='normal')),
]

regressionList = [
        ('KNN', KNeighborsRegressor(n_neighbors=3)),
        ('linear', LinearRegression()),
        ('randomForest', RandomForestRegressor(n_estimators=750)),
        ('SVM', SVR(C=399)),
]


def trainData(scaler, regression):
        model = Pipeline(
        [
        scaler,
        ('poly', PolynomialFeatures(
            degree=3,
            interaction_only=True,
            include_bias=False,
            )
        ),
        regression
        ]
        )
        model.fit(X_train, y_train)
        from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
        y_pred = model.predict(X_test)
        score = r2_score(y_test, y_pred)
        print(scaler[0], regression[0], end=' ')
        print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred, squared=False):.2f}')
        print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred):.2f}')
        print(f'R2 Score: {score:.4f}')
        
for regression in regressionList:
        for scaler in scalerList:
                trainData(scaler, regression)

RBscaler KNN Mean Squared Error: 2.91
Mean Absolute Error: 2.20
R2 Score: 0.8617
stdScaler KNN Mean Squared Error: 2.97
Mean Absolute Error: 2.35
R2 Score: 0.8562
scalerZ KNN Mean Squared Error: 2.62
Mean Absolute Error: 2.00
R2 Score: 0.8879
RBscaler linear Mean Squared Error: 2.58
Mean Absolute Error: 1.98
R2 Score: 0.8914
stdScaler linear Mean Squared Error: 2.58
Mean Absolute Error: 1.98
R2 Score: 0.8914
scalerZ linear Mean Squared Error: 2.77
Mean Absolute Error: 2.02
R2 Score: 0.8745


c:\Users\Administrator\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Administrator\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Administrator\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\preprocessing\_data.py:2762: UserWarning: n_quantiles (1000) is greater than the total number of samples (313). n_quantiles is set to n_samples.
  warnings.warn(
c:\Users\Administrator\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the

RBscaler randomForest Mean Squared Error: 2.41
Mean Absolute Error: 1.87
R2 Score: 0.9050


c:\Users\Administrator\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


stdScaler randomForest Mean Squared Error: 2.41
Mean Absolute Error: 1.80
R2 Score: 0.9053


c:\Users\Administrator\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Administrator\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\preprocessing\_data.py:2762: UserWarning: n_quantiles (1000) is greater than the total number of samples (313). n_quantiles is set to n_samples.
  warnings.warn(


scalerZ randomForest Mean Squared Error: 2.36
Mean Absolute Error: 1.90
R2 Score: 0.9090
RBscaler SVM Mean Squared Error: 2.92
Mean Absolute Error: 2.22
R2 Score: 0.8612
stdScaler SVM Mean Squared Error: 2.69
Mean Absolute Error: 2.09
R2 Score: 0.8823
scalerZ SVM Mean Squared Error: 3.12
Mean Absolute Error: 2.46
R2 Score: 0.8419


c:\Users\Administrator\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Administrator\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Administrator\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Administrator\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\preprocessing\_data.py:2762: UserWarning: n_quantiles (1000) is greater than

In [38]:

# for num in range(1,2):
model = Pipeline(
    [
        ('RBscaler', RobustScaler()),
        ('stdScaler', StandardScaler()),
        # ('scalerZ', QuantileTransformer(output_distribution='normal')),
        ('poly', PolynomialFeatures(
            degree=3,
            interaction_only=True,
            include_bias=False,
            )
        ),
        # ('KNN', KNeighborsRegressor(n_neighbors=3)),
        # ('linear', LinearRegression()),
        ('randomForest', RandomForestRegressor(n_estimators=750)),
        # ('SVM', SVR(C=399)),
        
    ]
)

model.fit(X_train, y_train)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = model.predict(X_test)
score = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred, squared=False):.2f}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred):.2f}')
print(f'R2 Score: {score:.4f}')
# if saved < score:
#     saved = score
#     saved_num = num

scores_train.append(model.score(X_train, y_train))
scores_test.append(model.score(X_test, y_test))
# print(f'Best num: {saved_num}, Best score: {saved}')


Mean Squared Error: 2.39
Mean Absolute Error: 1.82
R2 Score: 0.9069


c:\Users\Administrator\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [39]:
# plt.plot(range(len(scores_train)), scores_train, label='Training Score')
# plt.plot(range(len(scores_test)), scores_test, label='Test Score')
# plt.legend()
# plt.show()



In [40]:
# for idx, data in enumerate(scores_train):
#     tmp = scores_test[idx] - data
#     print(tmp, idx)